# DSPy
In this notebook, we'll be exploring a prompt optimization framework called DSPy.

## Notebook Setup

In [1]:
# Importing the necessary Python libraries
import os
import json

import numpy as np
import pandas as pd

In [2]:
# Loading our sample questions and answers associated to our knowledge item project
df_ki_qas = pd.read_csv('rag_sample_qas_from_kis.csv')

# Splitting the data into training and validation sets
df_train = df_ki_qas.sample(n = 7, random_state = 42)
df_val = df_ki_qas.drop(df_train.index)